<a href="https://colab.research.google.com/github/marcospiau/ia368-dd-dl4ir/blob/main/aula08-inpars/02_baseline_monot5_base_likelihood_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

I'm planning on trying to use the likelihood scoring evaluation already included in the T5 repository for re-ranking, instead of the Mesh-TensorFlow fork used by default in Pygaggle. If successful, it will be advantageous because it removes the need to maintain a fork of mesh-tensorflow, the default approach used by Pygaggle.

Before training any new models or developing new code using the new approach, we are going to validate the scoring and evaluation pipeline by modifying the original code from Castorini as little as possible.

# Hardware info

In [1]:
!echo *********
!echo lspu is "$(lscpu)"
# !echo *********
!echo free -mh is "$(free -mh)"
# !echo *********
!echo TPU_NAME is $TPU_NAME

sample_data
lspu is Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                        0
CPU MHz:                         2200.142
BogoMIPS:                        4400.28
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        256 KiB
L3 cache:                        55 MiB
NUMA node0 CPU(s):             

# Gcloud authentication

In [2]:
import os
from google.colab import auth
import tensorflow_gcs_config

os.environ['USE_AUTH_EPHEM'] = '0'
auth.authenticate_user(clear_output=False)

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=DkX7lca5U5gIXGEfbpS2nqoN7h5KID&prompt=consent&access_type=offline&code_challenge=NS3unbwcdSY0VyYIoq5dBiKamwiUy5sr5_ASBd4wZIo&code_challenge_method=S256

Enter authorization code: 4/0AbUR2VNusA9Lg0GtbUS1FNpaLAnUiqD4RUmwlwvpT6pnLVLe0yismCh7pvHS8F7y21XQEg


In [3]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=dhSlkwwlm3dVdB58LdShBWYGVYkeh1&prompt=consent&access_type=offline&code_challenge=bt58x1ipT6si7lqQZ7v7ihb2bm1W2KgHskbtIYXVcwA&code_challenge_method=S256

Enter authorization code: 4/0AbUR2VOgMdvG2uaktmjn6C0twh5IRFOpTE-Ap-PD3WQkcNObtFngDu0qTPGtr8YvM9kv8Q

You are now logged in as [m136825@dac.unicamp.br].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [5]:
!gcloud auth application-default login


The environment variable [GOOGLE_APPLICATION_CREDENTIALS] is set to:
  [/content/.adc/adc.json]
Credentials will still be generated to the default location:
  [/content/.config/application_default_credentials.json]
To use these credentials, unset this environment variable before
running your application.

Do you want to continue (Y/n)?  Y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=dzL23YIi5k8n3GrVBp7p4DDtlRA3bV&prompt=consent&access_type=offline&code_challenge=nrJ5jTUyYUJV6KVHygrXNVVsza_SokP1tPO4K0flqyQ&code_chal

In [2]:
BUCKET = 'gs://aula08-inpars'
os.environ['BUCKET'] = BUCKET
!gsutil ls {BUCKET}
!gsutil ls $BUCKET

gs://aula08-inpars/castorini_baseline/
gs://aula08-inpars/castorini_baseline/


# Installs

Anserini, pyserini and anserini-tools:

In [7]:
%%capture
!wget -nc https://raw.githubusercontent.com/marcospiau/ia368-dd-dl4ir/main/scripts/install_anserini.sh && chmod +x install_anserini.sh && time ./install_anserini.sh

Pygaggle:

In [8]:
!git clone --recursive https://github.com/castorini/pygaggle.git

Cloning into 'pygaggle'...
remote: Enumerating objects: 1562, done.
remote: Counting objects: 100% (634/634), done.
remote: Compressing objects: 100% (232/232), done.
remote: Total 1562 (delta 528), reused 431 (delta 402), pack-reused 928
Receiving objects: 100% (1562/1562), 513.12 KiB | 6.93 MiB/s, done.
Resolving deltas: 100% (1001/1001), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/pygaggle/tools'...
remote: Enumerating objects: 788, done.        
remote: Counting objects: 100% (545/545), done.        
remote: Compressing objects: 100% (467/467), done.        
remote: Total 788 (delta 101), reused 514 (delta 77), pack-reused 243        
Receiving objects: 100% (788/788), 119.60 MiB | 15.89 MiB/s, done.
Resolving deltas: 100% (185/185), done.
Submodule path 'tools': checked out '808f48711b5e172da6aec8b1855518c8ea65489f'


T5 (from pypi) and remaining stuff:

Vanilla stuff:

In [9]:
%%capture
!pip install -q ftfy polars toolz cytoolz transformers datasets dm-tree
!pip install -U t5[gcp,cache-tasks]==0.9.3
!pip install -U jaxlib
!sudo apt install -qq tree htop

# Imports

In [1]:
import os
import pandas as pd
import polars as pl
import ftfy
import datasets

import functools

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

os.environ['POLARS_FMT_STR_LEN']='1000'

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

os.environ['POLARS_FMT_STR_LEN']='1000'

# Preparing TREC-COVID data

Practically all data preparation was done on the first notebook, so we just need to pull some data from GCP for this one.

In [6]:
!mkdir -pv castorini_baseline
!gsutil -m rsync gs://aula08-inpars/castorini_baseline/ castorini_baseline/
!ls -lht castorini_baseline

Building synchronization state...
Starting synchronization...
total 271M
-rw-r--r-- 1 root root  1.3M May  2 04:53 castorini-baseline-monot5-base-predictions.txt-1100000
-rw-r--r-- 1 root root  185M May  2 04:30 trec-covid-corpus.tsv
-rw-r--r-- 1 root root   82M May  2 04:30 castorini-baseline-t5-input.txt
-rw-r--r-- 1 root root  1.8M May  2 04:30 run.title_and_text_no_expansion.txt
-rw-r--r-- 1 root root  578K May  2 04:30 castorini-baseline-t5-input_ids.txt
-rw-r--r-- 1 root root 1023K May  2 04:30 trec-covid-qrels_trec_format.txt
-rw-r--r-- 1 root root  3.6K May  2 04:30 trec-covid-queries_trec_format.txt


In [11]:
!head castorini_baseline/castorini-baseline-t5-input.txt
!wc -l castorini_baseline/castorini-baseline-t5-input.txt

Query: what is the origin of COVID-19 Document: [What is the origin of SARS-CoV-2?]. Every time a pandemic occurs, dozens of theories emerge to attribute the origin of the event to different facts. The COVID-19 pandemic that has hit virtually all the globe has been no exception. What is known so far about the origin of the virus that causes COVID 19? The first investigations on the origin of this disease have determined that it is a new type of virus, the origin of which is most likely zoonotic. Relevant:
Query: what is the origin of COVID-19 Document: Anesthesia and COVID-19: What We Should Know and What We Should Do.. Coronavirus disease 2019 (COVID-19), caused by Severe Acute Respiratory Syndrome-Coronavirus-2 (SARS-CoV-2), was first reported in Wuhan, Hubei, China, and has spread to more than 200 other countries around the world. COVID-19 is a highly contagious disease with continuous human-to-human transmission. The origin of the virus is unknown. Airway manipulations and intubati

In [16]:
def create_repeated_txt(output_file, n_repeats, str_to_repeat='true'):
    """Create a txt file with n_repeats of str_to_repeat.

    Args:
        output_file: the txt file to create
        n_repeats: the number of times to repeat str_to_repeat
        str_to_repeat: the string to repeat
    """
    with open(output_file, 'w') as f:
        for _ in range(n_repeats):
            f.write(str_to_repeat + '\n')

!mkdir -pv teste_score_eval
create_repeated_txt('teste_score_eval/true_50klines.txt', 50_000, 'true')
!ls -lht teste_score_eval/true_50klines.txt
!wc -l teste_score_eval/true_50klines.txt
!head teste_score_eval/true_50klines.txt

mkdir: created directory 'teste_score_eval'
-rw-r--r-- 1 root root 245K May  2 05:38 teste_score_eval/true_50klines.txt
50000 teste_score_eval/true_50klines.txt
true
true
true
true
true
true
true
true
true
true


In [79]:
!gsutil -m rsync teste_score_eval gs://aula08-inpars/teste_score_eval/
!gsutil ls gs://aula08-inpars/teste_score_eval/

Building synchronization state...
Starting synchronization...
Copying mtime from src to dst for gs://aula08-inpars/teste_score_eval/predictions.txt.lengths
Copying mtime from src to dst for gs://aula08-inpars/teste_score_eval/predictions.txt.targets
Copying mtime from src to dst for gs://aula08-inpars/teste_score_eval/predictions.txt.scores
Copying file://teste_score_eval/teste_score_eval_run.txt [Content-Type=text/plain]...
gs://aula08-inpars/teste_score_eval/predictions.txt.lengths
gs://aula08-inpars/teste_score_eval/predictions.txt.scores
gs://aula08-inpars/teste_score_eval/predictions.txt.targets
gs://aula08-inpars/teste_score_eval/teste_score_eval_run.txt
gs://aula08-inpars/teste_score_eval/true_50klines.txt


## Generating predictions

In [33]:
!wget -nc https://raw.githubusercontent.com/google-research/text-to-text-transfer-transformer/e0a69cd19d4e8b37332a0579715bbddb6e5e0830/t5/models/gin/score_from_file.gin
!cat score_from_file.gin

--2023-05-02 06:03:43--  https://raw.githubusercontent.com/google-research/text-to-text-transfer-transformer/e0a69cd19d4e8b37332a0579715bbddb6e5e0830/t5/models/gin/score_from_file.gin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 504 [text/plain]
Saving to: ‘score_from_file.gin’

score_from_file.gin 100%[===================>]     504  --.-KB/s    in 0s      

2023-05-02 06:03:43 (21.1 MB/s) - ‘score_from_file.gin’ saved [504/504]

# -*-Python-*-

inputs_filename = "/path/to/inputs"
targets_filename = "/path/to/targets"
scores_filename = "/path/to/scores"

utils.run.mode = "score_from_strings"
utils.get_variable_dtype.slice_dtype = "bfloat16"
utils.get_variable_dtype.activation_dtype = "bfloat16"

score_from_strings.inputs = %inputs_filename
score_from_stri

In [47]:
%%writefile monot5-base-score-eval.sh
set -u
export MODEL_NAME=base
export MODEL_DIR=gs://castorini/monot5/experiments/${MODEL_NAME}
export CHECKPOINT_STEP=1100000

ZONE=us-central1-f
TPU_SIZE=v2-8

INPUT_FILENAME=$1
OUTPUT_FILENAME=$2
TARGETS_FILENAME=$3
LOG_FILE=$4


# --gcp_project="${PROJECT_NAME}" \
# --gin_param="infer_model.decode_fn = @decode_from_file" \

time python3 -m t5.models.mesh_transformer_main \
--gin_file="gs://t5-data/pretrained_models/base/operative_config.gin" \
--gin_file="infer.gin" \
--tpu="${TPU_NAME}" \
--tpu_zone="${ZONE}" \
--model_dir="${MODEL_DIR}" \
--gin_param="utils.tpu_mesh_shape.tpu_topology = '${TPU_SIZE}'" \
--gin_file="beam_search.gin" \
--gin_param="infer_checkpoint_step = ${CHECKPOINT_STEP}" \
--gin_param="utils.run.sequence_length = {'inputs': 512, 'targets': 2}" \
--gin_param="Bitransformer.decode.max_decode_length = 2" \
--gin_param="utils.run.batch_size=('tokens_per_batch', 65536)" \
--gin_param="Bitransformer.decode.beam_size = 1" \
--gin_param="Bitransformer.decode.temperature = 0.0" \
--gin_param="Unitransformer.sample_autoregressive.sampling_keep_top_k = -1" \
--gin_file="score_from_file.gin" \
--gin_param="inputs_filename = '${INPUT_FILENAME}'" \
--gin_param="targets_filename = '${TARGETS_FILENAME}'" \
--gin_param="scores_filename = '${OUTPUT_FILENAME}'" \
2>&1 | tee -a $LOG_FILE

Overwriting monot5-base-score-eval.sh


In [48]:
!chmod +x monot5-base-score-eval.sh && \
time ./monot5-base-score-eval.sh \
gs://aula08-inpars/castorini_baseline/castorini-baseline-t5-input.txt \
gs://aula08-inpars/teste_score_eval/predictions.txt \
gs://aula08-inpars/teste_score_eval/true_50klines.txt \
teste_score_eval.log

2023-05-02 06:11:31.995155: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
non-resource variables are not supported in the long term
I0502 06:11:42.158062 140412500453184 mesh_transformer_main.py:171] No write access to model directory. Skipping command logging.
I0502 06:11:42.164677 140412500453184 resource_reader.py:50] system_path_file_exists:gs://t5-data/pretrained_models/base/operative_config.gin
E0502 06:11:42.165465 140412500453184 resource_reader.py:55] Path not found: gs://t5-data/pretrained_models/base/operative_config.gin
I0502 06:11:42.292235 140412500453184 config.py:2372] Skipping import of unknown module `t5.data.sentencepiece_vocabulary` (skip_unknown=True).
I0502 06:11:42.308861 140412500453184 resource_reader.py:50] system_path_file_exists:infer.gin
E0502 06:11:42.309308 140412500453184 resource_reader.py:55] Path not found: infer.gin
I0502 06:11:42.310658 140412500453184 resource_reader.py:50

It took 3m19.055s to score 50_000 pairs of queries and documents.

Getting files from GCP:

In [53]:
!gsutil -m rsync gs://aula08-inpars/teste_score_eval teste_score_eval
!ls -lht teste_score_eval
!wc -l teste_score_eval/*
!head teste_score_eval/*

Building synchronization state...
Starting synchronization...
Copying gs://aula08-inpars/teste_score_eval/predictions.txt.lengths...
Copying gs://aula08-inpars/teste_score_eval/predictions.txt.scores...
Copying gs://aula08-inpars/teste_score_eval/predictions.txt.targets...
/ [3/3 files][847.1 KiB/847.1 KiB] 100% Done                                    
Operation completed over 3 objects/847.1 KiB.                                    
total 1.1M
-rw-r--r-- 1 root root  98K May  2 06:14 predictions.txt.lengths
-rw-r--r-- 1 root root 245K May  2 06:14 predictions.txt.targets
-rw-r--r-- 1 root root 506K May  2 06:14 predictions.txt.scores
-rw-r--r-- 1 root root 245K May  2 05:38 true_50klines.txt
  50000 teste_score_eval/predictions.txt.lengths
  50000 teste_score_eval/predictions.txt.scores
  50000 teste_score_eval/predictions.txt.targets
  50000 teste_score_eval/true_50klines.txt
 200000 total
==> teste_score_eval/predictions.txt.lengths <==
2
2
2
2
2
2
2
2
2
2

==> teste_score_eval/predi

In [54]:
!python3 pygaggle/pygaggle/data/convert_run_from_t5_to_trec_format.py --help

usage: convert_run_from_t5_to_trec_format.py
       [-h]
       --predictions
       PREDICTIONS
       --query_run_ids
       QUERY_RUN_IDS
       --output
       OUTPUT

Convert T5
predictions
into a
TREC-
formatted
run.

options:
  -h, --help
    show this
    help
    message and
    exit
  --predictions PREDICTIONS
    T5
    predictions
    file.
  --query_run_ids QUERY_RUN_IDS
    File
    containing
    query doc
    id pairs
    paired with
    the T5's
    predictions
    file.
  --output OUTPUT
    run file in
    the TREC
    format.


Workaround to format our files into the ones pygaggle expects:

In [59]:
!paste -d'\t' teste_score_eval/predictions.txt.targets teste_score_eval/predictions.txt.scores | head

true	-0.00809574
true	-0.0049181
true	-0.0049181
true	-0.0321341
true	-0.0181508
true	-0.00975513
true	-0.00671673
true	-0.0133104
true	-0.00280571
true	-0.00671673


In [74]:
!python3 pygaggle/pygaggle/data/convert_run_from_t5_to_trec_format.py \
    --predictions=<(paste -d'\t' teste_score_eval/predictions.txt.targets teste_score_eval/predictions.txt.scores) \
    --query_run_ids=castorini_baseline/castorini-baseline-t5-input_ids.txt \
    --output=teste_score_eval/teste_score_eval_run.txt

!head teste_score_eval/teste_score_eval_run.txt
!tail teste_score_eval/teste_score_eval_run.txt
!ls -lht teste_score_eval/teste_score_eval_run.txt
!wc -l teste_score_eval/teste_score_eval_run.txt

Done!
1 Q0 4dtk1kyh 1 1.0 T5
1 Q0 k9lcpjyo 2 0.5 T5
1 Q0 pu9l36j9 3 0.3333333333333333 T5
1 Q0 nvofyg16 4 0.25 T5
1 Q0 rr7r7ef0 5 0.2 T5
1 Q0 wmfcey6f 6 0.16666666666666666 T5
1 Q0 utsr0zv7 7 0.14285714285714285 T5
1 Q0 kgifmjvb 8 0.125 T5
1 Q0 wco27nop 9 0.1111111111111111 T5
1 Q0 105q161g 10 0.1 T5
9 Q0 4x5tr0k5 991 0.0010090817356205853 T5
9 Q0 qxyk6gxt 992 0.0010080645161290322 T5
9 Q0 ioq50ntq 993 0.0010070493454179255 T5
9 Q0 320yujjc 994 0.001006036217303823 T5
9 Q0 0u5sym1n 995 0.0010050251256281408 T5
9 Q0 u6fii6u8 996 0.001004016064257028 T5
9 Q0 myjbk7ye 997 0.0010030090270812437 T5
9 Q0 m61jl8uf 998 0.001002004008016032 T5
9 Q0 rmhlrx1z 999 0.001001001001001001 T5
9 Q0 k4qor7n8 1000 0.001 T5
-rw-r--r-- 1 root root 2.1M May  2 06:29 teste_score_eval/teste_score_eval_run.txt
50000 teste_score_eval/teste_score_eval_run.txt


# Evaluating results

In [77]:
import shlex
import subprocess

TREC_EVAL_BIN_PATH = './tools/eval/trec_eval.9.0.4/trec_eval'
# Essa função foi escrita usando o github copilot
def get_trec_eval_metrics(flags, qrels_path, results_path):
    """Runs trec_eval and returns the results as a dictionary.

    Args:
        flags (str): Flags to pass to trec_eval.
        qrels_path (str): Path to the qrels file.
        results_path (str): Path to the results file.

    Returns:
        Dict[str, float]: A dictionary mapping metric names to their values.
    """
    output = subprocess.check_output([
        TREC_EVAL_BIN_PATH,
        qrels_path,
        results_path,
        *shlex.split(flags)
    ]).decode('utf-8')
    return {
        line.split()[0]: (line.split()[2])
        for line in output.splitlines()
    }


eval_metrics_fn = functools.partial(
    get_trec_eval_metrics,
    '-c -m ndcg_cut.10 -m recall.1000 -m recip_rank.10',
    'castorini_baseline/trec-covid-qrels_trec_format.txt'
)

In [78]:
eval_metrics_fn('teste_score_eval/teste_score_eval_run.txt')

{'recip_rank': '0.8585', 'recall_1000': '0.3955', 'ndcg_cut_10': '0.7174'}